In [1]:
#librerias
import pandas as pd
import numpy as np

In [ ]:
#https://sinergox.xm.com.co/hdrlg/Paginas/Historicos/Historicos.aspx
#https://sinergox.xm.com.co/trpr/Paginas/Historicos/Historicos.aspx?RootFolder=%2Ftrpr%2FHistricos%2FPrecios&FolderCTID=0x012000394993FA303733428C33EC91D1DFA6DB&View=%7B5CA2173E%2D1541%2D4EC7%2D9D1C%2DE145E3DFFAE3%7D

## Descarga de datos

In [2]:
#fechas: 	'1/04/2025'  y '9/09/2025'

#Precios declarados (daily bid prices op1)
archivo = "../datos/raw/PreciodeOfertaDeclarado.csv"
df_declarado = pd.read_csv(archivo, sep=";", encoding="utf-8")
df_declarado['Planta_Configuracion'] = df_declarado['CodigoPlanta'].astype(str) + "_" + df_declarado['Configuracion'].astype(str)

#Precio de oferta de la planta en el proceso del despacho ideal, en COP/kWh (daily bid prices op2)
archivo_oferta = "../datos/raw/Preciodeofertaplanta.csv"
df_oferta = pd.read_csv(archivo_oferta, sep=",", encoding="utf-8")


#Precio de bolsa horario (daily market clearing prices)
archivo_bolsa = "../datos/raw/Preciodebolsahorario.csv"
df_bolsa = pd.read_csv(archivo_bolsa, sep=",", encoding="utf-8")

#Disponibilidad Comercial por Planta (hourly-daily available capacity)
#https://www.simem.co/datadetail/24F4EC70-2DA0-4551-A188-FA750A7D66A5

julio = pd.read_csv("../datos/raw/Disponibilidadcomercialporplantajulio.csv", sep=',', encoding='utf-8')
septiembre = pd.read_csv("../datos/raw/Disponibilidadcomercialporplantaseptiembre.csv", sep=',', encoding='utf-8')
# Concatenar verticalmente
df_capacity = pd.concat([julio, septiembre], axis=0)
# Eliminar duplicados exactos
df_capacity = df_capacity.drop_duplicates()
df_capacity = df_capacity.reset_index(drop=True)



#Pronóstico del CND de demanda por Área operativa (daily aggregate demand)
archivo_pronostico= "../datos/raw/PronósticodelCNDdedemandaporÁreaoperativa.csv"
df_pronostico = pd.read_csv(archivo_pronostico, sep=",", encoding="utf-8")
df_pronostico_agg = (
    df_pronostico.groupby("FechaHora", as_index=False)["PronDem"]
      .sum()
      .rename(columns={"PronDem": "daily_agg_demand"})
)

# Despacho programado recursos de generación (cantidades de equilibrio)
#https://www.simem.co/datadetail/FF027B64-9896-4844-B827-9DC05F690668
archivo_despacho = "../datos/raw/Despachoprogramadorecursosdegeneracion.csv"
df_despacho = pd.read_csv(archivo_despacho, sep=",", encoding="utf-8")
df_despacho_agg = (
    df_despacho.groupby("FechaHora", as_index=False)["Valor"]
      .sum()
      .rename(columns={"Valor": "daily_eq_demand"})
)

#Resrvas hidraulicas
#https://www.simem.co/datadetail/9605B04C-8D3A-40A9-94D3-5535670D2F9A
archivo_hidraulica = "../datos/raw/ReservasHidráulicas.csv"
df_hidraulica = pd.read_csv(archivo_hidraulica, sep=",", encoding="utf-8")

df_hidraulica_avg= (
    df_hidraulica
    .groupby('Fecha', as_index=False)[['VolumenUtilDiarioMasa', 'VolumenTotalMasa']]
    .mean()
)

## Clusterizacion

In [3]:
from sklearn.cluster import KMeans
import pandas as pd

# 1. Seleccionar la variable
X = df_hidraulica_avg[['VolumenUtilDiarioMasa']].values

# 2. Aplicar K-Means con 12 grupos
kmeans = KMeans(n_clusters=12, random_state=42)
df_hidraulica_avg['cluster'] = kmeans.fit_predict(X)

# 3. Reordenar los clusters según el valor medio (1 = menor, 12 = mayor)
cluster_means = (
    df_hidraulica_avg.groupby('cluster')['VolumenUtilDiarioMasa']
    .mean()
    .sort_values()
)
orden_map = {old: new + 1 for new, old in enumerate(cluster_means.index)}
df_hidraulica_avg['cluster'] = df_hidraulica_avg['cluster'].map(orden_map)

# 4. Verificar
print(df_hidraulica_avg[['Fecha', 'VolumenUtilDiarioMasa', 'cluster']].head())

# 5. Resumen por cluster
resumen = (
    df_hidraulica_avg.groupby('cluster')['VolumenUtilDiarioMasa']
    .agg(['count', 'mean', 'min', 'max'])
    .sort_index()
)
print(resumen)

        Fecha  VolumenUtilDiarioMasa  cluster
0  2025-04-01           2.969186e+08        1
1  2025-04-02           2.970081e+08        1
2  2025-04-03           2.971947e+08        1
3  2025-04-04           2.981375e+08        1
4  2025-04-05           3.019760e+08        1
         count          mean           min           max
cluster                                                 
1           24  3.099078e+08  2.969186e+08  3.217660e+08
2            5  3.377511e+08  3.241564e+08  3.503973e+08
3            6  3.670548e+08  3.582817e+08  3.764841e+08
4           13  3.950024e+08  3.882780e+08  4.060557e+08
5           14  4.239925e+08  4.119788e+08  4.286237e+08
6           37  4.384967e+08  4.323997e+08  4.414406e+08
7           12  4.456888e+08  4.423426e+08  4.487055e+08
8           25  4.548213e+08  4.503221e+08  4.596065e+08
9           13  4.644893e+08  4.600609e+08  4.722815e+08
10           8  8.354724e+08  8.315556e+08  8.396919e+08
11           4  8.455813e+08  8.409993e+

## Merges

In [4]:
# Renombrar columnas antes del merge
df_oferta_renamed = df_oferta[['FechaHora', 'CodigoPlanta', 'Valor']].rename(columns={'Valor': 'precio'})
df_capacity_renamed = df_capacity[['FechaHora','Valor', 'CodigoPlanta']].rename(columns={'Valor': 'cantidad'})

df_merged = df_oferta_renamed.merge(
    df_capacity_renamed,
    on=['FechaHora', 'CodigoPlanta'],
    how='left'
)

# Reemplazar NaN en cantidad por 0
df_merged['cantidad'] = df_merged['cantidad'].fillna(0)
df_merged = df_merged.drop_duplicates(subset=['FechaHora', 'CodigoPlanta'], keep='first')

In [5]:
df_merged['Fecha'] = pd.to_datetime(df_merged['FechaHora']).dt.date
df_hidraulica_avg['Fecha'] = pd.to_datetime(df_hidraulica_avg['Fecha']).dt.date

In [6]:
df_final = pd.merge(
    df_merged,
    df_hidraulica_avg[['Fecha', 'cluster']],
    on='Fecha',
    how='inner'
)
df_final

,FechaHora,CodigoPlanta,precio,cantidad,Fecha,cluster
0,2025-05-25 23:00:00,TEC1,1486.46,213000.0,2025-05-25,5
1,2025-05-25 22:00:00,TEC1,1486.46,213000.0,2025-05-25,5
2,2025-05-25 21:00:00,TEC1,1486.46,213000.0,2025-05-25,5
3,2025-05-25 20:00:00,TEC1,1486.46,213000.0,2025-05-25,5
4,2025-05-25 19:00:00,TEC1,1486.46,213000.0,2025-05-25,5
...,...,...,...,...,...,...
307147,2025-07-13 04:00:00,TGJ1,557.16,0.0,2025-07-13,9
307148,2025-07-13 03:00:00,TGJ1,557.16,0.0,2025-07-13,9
307149,2025-07-13 02:00:00,TGJ1,557.16,0.0,2025-07-13,9
307150,2025-07-13 01:00:00,TGJ1,557.16,0.0,2025-07-13,9


In [7]:
df_bolsa

,CodigoVariable,FechaHora,CodigoDuracion,UnidadMedida,Version,Valor
0,PB_Int,2025-06-01 00:00:00,PT1H,COP/kWh,TX2,193.75
1,PB_Int,2025-06-01 01:00:00,PT1H,COP/kWh,TX2,193.75
2,PB_Int,2025-06-01 02:00:00,PT1H,COP/kWh,TX2,193.75
3,PB_Int,2025-06-01 03:00:00,PT1H,COP/kWh,TX2,193.75
4,PB_Int,2025-06-01 04:00:00,PT1H,COP/kWh,TX2,193.75
...,...,...,...,...,...,...
59443,PB_Tie,2025-05-18 19:00:00,PT1H,COP/kWh,TXF,106.39
59444,PB_Tie,2025-05-18 20:00:00,PT1H,COP/kWh,TXF,106.39
59445,PB_Tie,2025-05-18 21:00:00,PT1H,COP/kWh,TXF,106.39
59446,PB_Tie,2025-05-18 22:00:00,PT1H,COP/kWh,TXF,102.64


In [8]:
#transado en el cierre

df_transado = pd.merge(
    df_bolsa[['FechaHora', 'Valor']],
    df_despacho_agg[['FechaHora', 'daily_eq_demand']],
    on='FechaHora',
    how='left'
)
df_transado = df_transado.rename(columns={
    'Valor': 'precio',
    'daily_eq_demand': 'demanda'
})

In [9]:
df_despacho_agg[df_despacho_agg['FechaHora']=='2025-05-25 23:00:00']

,FechaHora,daily_eq_demand
1319,2025-05-25 23:00:00,8524260


In [10]:
df_bolsa[df_bolsa['FechaHora']=='2025-05-25 23:00:00']

,CodigoVariable,FechaHora,CodigoDuracion,UnidadMedida,Version,Valor
49535,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,103.00
49559,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,108.56
49583,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,103.00
49607,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,105.39
49631,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,111.04
49655,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,105.39
49679,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,103.00
49703,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,106.56
49727,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,103.00
49751,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TXR,105.39


In [11]:
# Asegurarse de que las columnas 'FechaHora' sean datetime
df_final['FechaHora'] = pd.to_datetime(df_final['FechaHora'])
df_transado['FechaHora'] = pd.to_datetime(df_transado['FechaHora'])

# Filtrar df_transado: solo fechas que aparecen en df_final
df_transado_filtrado = df_transado[df_transado['FechaHora'].isin(df_final['FechaHora'])]

# Filtrar df_final: solo fechas que aparecen en df_transado_filtrado
df_final_filtrado = df_final[df_final['FechaHora'].isin(df_transado_filtrado['FechaHora'])]

## Guardar resultados

In [12]:
df_final.to_csv("../datos/procesado/df_final.csv", index=False)
df_transado.to_csv("../datos/procesado/df_transado.csv", index=False)
df_despacho_agg.to_csv("../datos/procesado/df_despacho_agg.csv", index=False)